In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense, TextVectorization, Embedding, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import ast

2024-06-08 22:49:33.623225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 22:49:33.623287: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 22:49:33.623343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-08 22:49:33.686759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-06-08 22:49:36.694625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:36.711874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:36.711930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
df = pd.read_csv("ner.csv", encoding = "unicode_escape")
df

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"['JJ', 'NN', 'NN', 'NNS', 'VBP', 'VBG', 'PRP$'...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"['JJ', 'NNS', 'VBD', 'DT', 'NN', 'VBD', 'VBN',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"['CD', 'JJR', 'VBD', 'IN', 'NNS', 'VBG', 'TO',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"['PRP', 'VBP', 'RB', 'DT', 'IN', 'DT', 'NNS', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [4]:
df = df[["Sentence", "Tag"]]
df

,Sentence,Tag
0,Thousands of demonstrators have marched throug...,"['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Families of soldiers killed in the conflict jo...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,They marched from the Houses of Parliament to ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"Police put the number of marchers at 10,000 wh...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,The protest comes on the eve of the annual con...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...
47954,Indian border security forces are accusing the...,"['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe..."
47955,Indian officials said no one was injured in Sa...,"['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
47956,Two more landed in fields belonging to a nearb...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
47957,They say not all of the rockets exploded upon ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [5]:
df["Sentence"].to_list()[10]

'The step will allow the facility to operate at full capacity .'

In [6]:
df["Tag"].to_list()[10]

"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"

In [7]:
df_train, df_test = train_test_split(df, test_size=0.2)
print(f"Train size: {len(df_train)} \nTest size: {len(df_test)}")

Train size: 38367 
Test size: 9592


In [8]:
df["Tag"]

0        ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...
1        ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
2        ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
3        ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
4        ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
                               ...                        
47954    ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe...
47955    ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...
47956    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
47957    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...
47958         ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Name: Tag, Length: 47959, dtype: object

In [9]:
unique_tags = set( [tag for sub_tag in list(df.Tag.values) for tag in ast.literal_eval(sub_tag)] )
unique_tags.add("[START]")
unique_tags.add("[END]")
unique_tags.add("[PAD]")
unique_tags

{'B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O',
 '[END]',
 '[PAD]',
 '[START]'}

In [10]:
le = LabelEncoder()
le.fit(list(unique_tags))


LabelEncoder()

In [11]:
vector_size = 128
max_length = 100
K = 18

In [12]:
encoded_tags_train = []

for tags in df_train["Tag"].to_list():
    lst = ast.literal_eval(tags)
    lst.insert(0, "[START]")
    lst.append("[END]")
    encoded_tags_train.append(le.transform(lst))

encoded_tags_test = []

for tags in df_test["Tag"].to_list():
    lst = ast.literal_eval(tags)
    lst.insert(0, "[START]")
    lst.append("[END]")
    encoded_tags_test.append(le.transform(lst))


train_targets_final = pad_sequences(encoded_tags_train,maxlen=max_length+1,padding="post", value = 18)
print("Shape of train targets: ",train_targets_final.shape)

test_targets_final = pad_sequences(encoded_tags_test,maxlen=max_length+1,padding="post", value = 18)
print("Shape of test targets: ",test_targets_final.shape)

Shape of train targets:  (38367, 101)
Shape of test targets:  (9592, 101)


In [13]:
(encoded_tags_train)

[array([19, 16,  2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  3, 16, 16,
        16, 16, 16,  2, 10, 16, 17]),
 array([19, 16, 16, 16, 16, 16,  2, 16, 16, 16, 16, 16, 16, 16, 16, 16,  2,
        16, 16, 16, 16,  2, 16, 16, 16, 16, 16, 16, 16, 16, 17]),
 array([19, 16, 16, 16,  2,  7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17]),
 array([19,  2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  6,
        14, 16, 16, 16, 16, 16, 16, 17]),
 array([19, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 17]),
 array([19, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16,  5, 13, 13, 13, 16, 16, 16, 16, 16, 16, 16, 16,  7,
        15, 16,  7, 16, 17]),
 array([19, 16,  7, 16, 16, 16, 16, 16, 16,  2, 16, 16, 16, 16, 16,  2, 16,
        16, 16,  7, 16, 16, 16, 16, 16, 16, 16, 17]),
 array([19, 16,  5, 16,  7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16

In [14]:
df["Sentence"].values

array(['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
       'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
       'They marched from the Houses of Parliament to a rally in Hyde Park .',
       ..., 'Two more landed in fields belonging to a nearby village .',
       'They say not all of the rockets exploded upon impact .',
       'Indian forces said they responded to the attack'], dtype=object)

In [15]:
def positional_encoding( max_len, d_model):
    d_model /= 2
    pos = np.arange(max_len)[:, np.newaxis]
    dims = np.arange(d_model)[np.newaxis, :]/d_model
    frequency = pos * ( 1 / ( 10000 ** dims) )


    pos_encoding = np.concatenate( [np.sin(frequency), np.cos(frequency)], axis=-1)
    return tf.cast( pos_encoding, dtype = tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self,vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.positional_encoding = positional_encoding(max_length, d_model)
        self.embedding = tf.keras.layers.Embedding( vocab_size, d_model, mask_zero = True )

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
    
    def call(self, x):
        max_length = tf.shape(x)[1]
        x = self.embedding(x)
        x *= tf.math.sqrt( tf.cast(self.d_model, tf.float32) )
        x = x + self.positional_encoding[ tf.newaxis, :max_length, :]
        return x

In [16]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, dense_hidden_num_units , d_model):
        super().__init__()
        self.ffn = tf.keras.Sequential(
            [
                Dense(dense_hidden_num_units, activation="relu"),
                Dense(d_model)
            ]
        )
        self.add = Add()
        self.layernorm = LayerNormalization()
    
    def call( self, x):
        x = self.add([x, self.ffn(x)])
        x = self.layernorm(x)
        return x

class BaseAttention( tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = MultiHeadAttention(**kwargs)
        self.layernorm = LayerNormalization()
        self.add = Add()
    
class EncoderSelfAttention(BaseAttention):
    def call(self, x):
        mha_out = self.mha( query = x, key = x, value = x)
        x = self.add([x, mha_out])
        x = self.layernorm(x)
        return x
    

class CrossAttention(BaseAttention):
    def call(self, x, context):
        mha_out = self.mha( query = x, key= context, value = context)
        x = self.add([x, mha_out])
        x = self.layernorm(x)
        return x

class CausalSelfAttention(BaseAttention):
    def call(self, x):
        mha_out = self.mha( query = x, key = x, value = x, use_causal_mask = True)
        x = self.add([x, mha_out])
        x = self.layernorm(x)
        return x


In [17]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_model, dense_hidden_num_units):
        super().__init__()
        self.self_attention = EncoderSelfAttention( num_heads = num_heads, key_dim = d_model)
        self.ffn = FeedForward(dense_hidden_num_units, d_model)
    
    def call(self,x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

In [18]:
class Encoder( tf.keras.layers.Layer ):
    def __init__(self, num_heads, d_model, num_layers, vocab_size, dense_hidden_num_units ):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.positional_embedding = PositionalEmbedding(vocab_size, d_model)
        
        self.encoder_layers = []
        for i in range(num_layers):
            self.encoder_layers.append( EncoderLayer( num_heads, d_model, dense_hidden_num_units ) )
    
    def call(self,x):
        x = self.positional_embedding(x)
        for i in range(self.num_layers):
            x = self.encoder_layers[i](x)
        return x

In [19]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dense_hidden_num_units):
        super().__init__()
        self.causal_self_attention = CausalSelfAttention( num_heads = num_heads, key_dim = d_model)
        self.cross_attention = CrossAttention( num_heads = num_heads, key_dim = d_model )
        self.ffn = FeedForward(dense_hidden_num_units, d_model)
    
    def call(self, x, context):
        x = self.causal_self_attention(x)
        x = self.cross_attention(x, context)
        x = self.ffn(x)
        return x

In [20]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dense_hidden_num_units, vocab_size):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.positional_embedding = PositionalEmbedding( vocab_size, d_model )
        self.decoder_layers = []
        for i in range(num_layers):
            self.decoder_layers.append(
                DecoderLayer(d_model, num_heads, dense_hidden_num_units)
            )
    
    def call(self, x, context):
        x = self.positional_embedding(x)
        for i in range(self.num_layers):
            x = self.decoder_layers[i](x, context)
        return x

In [21]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dense_hidden_num_units, input_vocab_size, target_vocab_size):
        super().__init__()
        self.encoder = Encoder( num_heads, d_model, num_layers, input_vocab_size, dense_hidden_num_units)
        self.decoder = Decoder( num_layers, d_model, num_heads, dense_hidden_num_units, target_vocab_size)

        self.final_layer = Dense(target_vocab_size)

    def call(self, inputs):
        context, x = inputs
        context = self.encoder(context)
        x = self.decoder(x, context)
        output = self.final_layer(x)
        return output

In [22]:
data_train = tf.data.Dataset.from_tensor_slices( (df_train["Sentence"].values,train_targets_final))

data_test = tf.data.Dataset.from_tensor_slices( (df_test["Sentence"].values,test_targets_final))

2024-06-08 22:49:41.855295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:41.855426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:41.855460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:42.031677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-08 22:49:42.031746: I tensorflow/compile

In [23]:

vectorizer = TextVectorization( output_sequence_length=max_length, standardize="lower", output_mode="int")
vectorizer.adapt( df_train["Sentence"] )


In [24]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [25]:
def prepare_batch(sentences, tags):
    return ( vectorizer(sentences), tags[:,:-1] ), tags[:,1:] 

def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

In [26]:
train_batches = make_batches(data_train)
test_batches = make_batches(data_test)

In [27]:
def decode( vector):
    vocab = vectorizer.get_vocabulary()
    return( " ".join([vocab[each] for each in tf.squeeze(vector)]) )

for (sentence, tag), tag_labels in train_batches.take(1):
    break
tag_labels[1]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  6, 16, 16,
       16, 16, 16, 16,  2, 16, 16, 16, 16, 16,  7, 16, 17, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18],
      dtype=int32)>

In [28]:
num_layers = 2
d_model = 64
dense_hidden_num_units = 256
num_heads = 8

transformer = Transformer(num_layers, d_model, num_heads, dense_hidden_num_units, vectorizer.vocabulary_size(), 20)

transformer.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)



In [29]:
vectorizer(df_train["Sentence"].to_list()[6])

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([    5, 10544,     4,   272,     2, 11051,   127,     4,   249,
         729,     8,  5296,     6,  6121,   337,     4,     9,     5,
        8166,    29,    19,  3812,    25,     8,  3166,     3,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])>

In [30]:
history = transformer.fit(
    train_batches,
    epochs=5,
    validation_data=test_batches
)

Epoch 1/5


2024-06-08 22:49:51.267296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-06-08 22:49:51.392798: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f707804d890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-08 22:49:51.392855: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-06-08 22:49:51.401680: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-08 22:49:51.482071: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


600/600 [==============================] - 78s 112ms/step - loss: 0.1108 - accuracy: 0.9698 - val_loss: 0.0367 - val_accuracy: 0.9887
Epoch 2/5
600/600 [==============================] - 58s 96ms/step - loss: 0.0251 - accuracy: 0.9923 - val_loss: 0.0250 - val_accuracy: 0.9925
Epoch 3/5
600/600 [==============================] - 58s 96ms/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.0240 - val_accuracy: 0.9928
Epoch 4/5
600/600 [==============================] - 57s 96ms/step - loss: 0.0127 - accuracy: 0.9959 - val_loss: 0.0258 - val_accuracy: 0.9926
Epoch 5/5
600/600 [==============================] - 58s 96ms/step - loss: 0.0102 - accuracy: 0.9966 - val_loss: 0.0285 - val_accuracy: 0.9924


In [31]:
(df_test["Tag"].to_list()[5])

"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O']"

In [32]:
for (s,t), t_l in train_batches.take(1):
    break
decode(s[1])

'greenland was granted self-government in 1979 by the danish parliament ; the law went into effect the following year .                                                                                '

In [33]:
class NamedEntityRecognizer(tf.Module):
  def __init__(self, vectorizer, transformer):
    self.vectorizer = vectorizer
    self.transformer = transformer

  def __call__(self, sentence, max_length=max_length):
    sentence = vectorizer(sentence)

    encoder_input = sentence

    start_end = le.transform(["[START]","[END]"])
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      predictions = predictions[:, -1:, :]  
      predicted_id = tf.argmax(predictions, axis=-1)

      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    text = [le.inverse_transform(out) for out in output]

    return text


In [34]:
translator = NamedEntityRecognizer(vectorizer, transformer)

In [35]:
sentence = [decode(s[4])]

translated_text = translator(
    sentence
)
translated_text

[array(['[START]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O',
        'B-geo', 'I-geo', 'O', '[END]'], dtype='<U7')]

In [38]:
sentence

['he also said that his government is studying more than 30 road-building projects in iran to further expand trade to afghanistan and central asia .                                                                           ']

In [37]:
le.inverse_transform(t[4])

array(['[START]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O',
       'B-geo', 'I-geo', 'O', '[END]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], dtype='<U7')